<a href="https://colab.research.google.com/github/GasanaElysee12/DNA-Classification-with-kernel-methods/blob/main/DNA_PROJECT_FINAL_PART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let start by initializing the required python Libraries in this work We will be classifiying whether the input sequence of DNA has CovID-19 or not.

In [ ]:
import numpy as np
import pandas as pd
import cvxopt
from cvxopt import matrix
import cvxpy as cp
import scipy.sparse as sparse
from tqdm import tqdm
import pickle


# Reading of the data and make ID index by dropping the index column.

In [ ]:
X_train = pd.read_csv("/content/Xtr.csv", sep=",", index_col=0)
X_test = pd.read_csv("/content/Xte.csv", sep=",", index_col=0)
Y_train = pd.read_csv("/content/Ytr.csv", sep=",", index_col=0)

# Let's check what are in the datasets

In [ ]:
# Training example
X_train.head()

,Sequence
Id,
1,TCTTCCATCGTTGATAGTGTTACATTGAAGAATGCGACGATCCATC...
2,CTTCTTCAGTAATTACCTCAAGAGACGTTTAGTCTCTAATGGTGTT...
3,GTCTATGGTGCTGCTGTTGTTTACCGGGGTACAACAACTTACAAAT...
4,ATTCGCCATGAGTAAATTTCCCCTTAAATTAGGGGGTTCAGCTGTT...
5,CAGACAAAACATTTGTGTCTGGTAACTGTGATGTTGTAATAGGAAT...


In [ ]:
#Test example
X_test.head()

,Sequence
Id,
1,GGAGCATATCTGGCAGGCCTGGTGCCGGCGCCATGGCGTGGGCTGG...
2,ACTGTTTTACCACCTTTTCTCACAGATGACATGTTTGCTCCATACA...
3,TTGTTAGATTTCTTAATATTACAAAGTTGTGCACTTTTGGTGAAGT...
4,ACCTTCGCCATCAGCCTGCGCCCTTCATCGGCGCGTCACTGGTTAA...
5,ATTCACAGACCATTCCAGGAGCAGTGACAATATTGCTTTGCATGAA...


In [ ]:
#y_train example
Y_train.head()

,Covid
Id,
1,1
2,1
3,1
4,1
5,1


# Convert  the data into numpy array values

In [ ]:

X_train= X_train.values
X_test = X_test.values
Y_train = Y_train.values

AttributeError: ignored

# In the following cell, this is the parent class of kernel that will be used to initialize similarity, creating n-grams and return matrix of similarity.

In [ ]:


class Kernel():
    """ Abstract Kernel class"""

    def __init__(self):
        pass

    def similarity(self, x, y):
        """ Similarity between 2 feature vectors (depends on the type of kernel)"""
        return -1

    def gram(self, X1, X2=None):
        """ Compute the gram matrix of a data vector X where the (i,j) entry is defined as <Xi,Xj>\\
        X1: data vector (n_samples_1 x n_features)
        X2: data vector (n_samples_2 x n_features), if None compute the gram matrix for (X1,X1)
        """
        if X2 is None:
            X2=X1
        n_samples_1 = X1.shape[0]
        n_samples_2 = X2.shape[0]
        G = np.zeros((n_samples_1, n_samples_2))
        for ii in tqdm(range(n_samples_1)):
            for jj in range(n_samples_2):
                G[ii,jj] = self.similarity(X1[ii], X2[jj])
        return G

# Down, this is the implementation of the Mismatch Kernel that will help us to detect mismatch based on the similarity between two sequences example

In [ ]:
class MismatchKernel(Kernel):

    def __init__(self, k, m, neighbours, kmer_set, normalize=False):
        super().__init__()
        self.k = k
        self.m = m
#         self.neigh_kmer=None
        self.kmer_set = kmer_set #kmer_set and neighbours have to be pre-computed (to save computational time when running multiple experiments)
        self.neighbours = neighbours
        self.normalize = normalize

    def neighbour_embed_kmer(self, x):
        """
        Embed kmer with neighbours.
        x: str
        """
        kmer_x = [x[j:j + self.k] for j in range(len(x) - self.k + 1)]
        x_emb = {}
        for kmer in kmer_x:
#


            neigh_kmer = self.neighbours[kmer]


            for neigh in neigh_kmer:
                idx_neigh = self.kmer_set[neigh]
                if idx_neigh in x_emb:
                    x_emb[idx_neigh] += 1
                else:
                    x_emb[idx_neigh] = 1
        return x_emb


    def neighbour_embed_data(self, X):
        """
        Embed data with neighbours.
        X: array of string
        """
        X_emb = []
        for i in range(len(X)):
            x = X[i]
            x_emb = self.neighbour_embed_kmer(x)
            X_emb.append(x_emb)
        return X_emb

    def to_sparse(self, X_emb):
        """
        Embed data to sparse matrix.
        X_emb: list of dict.
        """
        data, row, col = [], [], []
        for i in range(len(X_emb)):
            x = X_emb[i]
            data += list(x.values())
            row += list(x.keys())
            col += [i for j in range(len(x))]
        X_sm = sparse.coo_matrix((data, (row, col)))
        return X_sm

    def similarity(self, x, y):
        """ Mismatch kernel \\
        x, y: are string
        """
        x_emb = self.neighbour_embed_kmer(x)
        y_emb = self.neighbour_embed_kmer(y)
        sp = 0
        for idx_neigh in x_emb:
            if idx_neigh in y_emb:
                sp += x_emb[idx_neigh] * y_emb[idx_neigh]
        if self.normalize:
            sp /= np.sqrt(np.sum(np.array(list(x_emb.values()))**2))
            sp /= np.sqrt(np.sum(np.array(list(y_emb.values()))**2))
        return sp

    def gram(self, X1, X2=None):
        """ Compute the gram matrix of a data vector X where the (i,j) entry is defined as <Xi,Xj>\\
        X1: array of string (n_samples_1,)
        X2: array of string (n_samples_2,), if None compute the gram matrix for (X1,X1)
        """

        X1_emb = self.neighbour_embed_data(X1)
        X1_sm = self.to_sparse(X1_emb)

        if X2 is None:
            X2 = X1
        X2_emb = self.neighbour_embed_data(X2)
        X2_sm = self.to_sparse(X2_emb)

        # Reshape matrices if the sizes are different
        nadd_row = abs(X1_sm.shape[0] - X2_sm.shape[0])
        if X1_sm.shape[0] > X2_sm.shape[0]:
            add_row = sparse.coo_matrix(([0], ([nadd_row-1], [X2_sm.shape[1]-1])))
            X2_sm = sparse.vstack((X2_sm, add_row))
        elif X1_sm.shape[0] < X2_sm.shape[0]:
            add_row = sparse.coo_matrix(([0], ([nadd_row - 1], [X1_sm.shape[1] - 1])))
            X1_sm = sparse.vstack((X1_sm, add_row))

        G = (X1_sm.T * X2_sm).todense().astype('float')

        if self.normalize:
            G /= np.array(np.sqrt(X1_sm.power(2).sum(0)))[0,:,None]
            G /= np.array(np.sqrt(X2_sm.power(2).sum(0)))[0,None,:]

        return G

# Down this is the SVM model that will be used as classifier:

In [ ]:


class SVM():
    """
    SVM implementation

  Method that will be used and its arguments:
        svm = SVM(kernel='linear', C=1)
        svm.fit(X_train, y_train)
        svm.predict(X_test)
    """

    def __init__(self, kernel, C=1.50, tol_support_vectors=1e-4):
        """
        kernel: it represents the kernel to use
        C: float > 0, default=1.0, regularization parameter for error once we have non linear
           separable data
        tol_support_vectors: Is the threshold alpha value for considering vectors as support vectors
        """
        self.kernel = kernel
        self.C = C
        self.tol_support_vectors = tol_support_vectors

    def fit(self, X, y):

        self.X_train = X
        n_samples = X.shape[0]
        print("Computing the kernel Start")
        self.X_train_gram = self.kernel.gram(X)
        print("Computation is done!")

        #The following matrix will help to define the optimization problem that we have tosolve

        P = self.X_train_gram
        q = -y.astype('float')
        G = np.block([[np.diag(np.squeeze(y).astype('float'))],[-np.diag(np.squeeze(y).astype('float'))]])
        h = np.concatenate((self.C*np.ones(n_samples),np.zeros(n_samples)))

        # cvxopt will help to Solve the problem, using quadratic programming


        P=matrix(P)
        q=matrix(q)
        G=matrix(G)
        h=matrix(h)
        solver = cvxopt.solvers.qp(P=P,q=q,G=G,h=h)
        x = solver['x']
        self.alphas = np.squeeze(np.array(x))

        #Retrieve the support vectors
        self.support_vectors_indices = np.squeeze(np.abs(np.array(x))) > self.tol_support_vectors
        self.alphas = self.alphas[self.support_vectors_indices]
        self.support_vectors = self.X_train[self.support_vectors_indices]

        print(len(self.support_vectors), "support vectors out of",len(self.X_train), "training samples")

        return self.alphas


    def predict(self, X):
        """
        X: Is the array of shape (n_samples, n_features)\\
        Return: float array (n_samples,)
        """
        K = self.kernel.gram(X, self.support_vectors)
        y = np.dot(K, self.alphas)
        return y

    def predict_classes(self, X, threshold=0):
        """
        X: Is array of the shape (n_samples, n_features)\\
        Return: 0 and 1 array (n_samples,)
        """
        K = self.kernel.gram(X, self.support_vectors)
        y = np.dot(K, self.alphas)
        return np.where(y > threshold, 1, -1)

* Since the data are long sequences and have different length based on the row
in the dataframe, let cut them into small fragment of length $k$ and allow maximum
mismatching of $m$.
then these subsequences will be called "kmer".

* The following cell contains the functions that will be used to create kmer-set
that will help to get vocabular or set of unique kmers that will help
to build small matrix, others for detecting the neighbors of kmers up to mismatch.

* Also others will help to input dataset, length of kmer and maximum allowed  mismatch and returns the set of kmer and its neighbors


In [ ]:
def create_kmer_set(X, k, kmer_set={}):
    """
    Return a set of all kmers appearing in the dataset.
    """
    len_seq = len(X[0])
    idx = len(kmer_set)
    for i in range(len(X)):
        x = X[i]
        kmer_x = [x[i:i + k] for i in range(len_seq - k + 1)]
        for kmer in kmer_x:
            if kmer not in kmer_set:
                kmer_set[kmer] = idx
                idx += 1
    return kmer_set


def m_neighbours(kmer, m, recurs=0):
    """
    This function will return a list of neighbours kmers (up to m mismatches).
    """
    if m == 0:
        return [kmer[:-1]]

    letters = ['G', 'T', 'A', 'C']
    k = len(kmer[:-1])
    neighbours = m_neighbours(kmer[:-1], m - 1, recurs + 1)

    for j in range(len(neighbours)):
        neighbour = neighbours[j]
        for i in range(recurs, k - m + 1):
            for l in letters:
                neighbours.append(neighbour[:i] + l + neighbour[i + 1:])
    return list(set(neighbours))


def get_neighbours(kmer_set, m):
    """
    Find the neighbours given a set of kmers.
    """
    kmers_list = list(kmer_set.keys())
    kmers = np.array(list(map(list, kmers_list)))

    num_kmers, kmax = kmers.shape
    neighbours = {}
    for i in range(num_kmers):
        neighbours[kmers_list[i]] = []

    for i in tqdm(range(num_kmers)):
        kmer = kmers_list[i]
        kmer_neighbours = m_neighbours(kmer[:-1], m)
        for neighbour in kmer_neighbours:
            if neighbour in kmer_set:
                neighbours[kmer].append(neighbour)
    return neighbours


def load_neighbors(dataset, k, m):
    """
    dataset: is the sequence to be cutted into kmer\\
    k: is the length of the kmers
    m: is the number of possible (maximum aloowed) mismatches
    """
    file_name = 'neighbours_'+str(dataset)+'_'+str(k)+'_'+str(m)+'.p'
    # Load
    neighbour, kmer_set = pickle.load(open(file_name, 'rb'))
    print('Neighbors correctly loaded!')
    return neighbour, kmer_set


def load_or_compute_neighbors(dataset,k,m):
    """
    dataset: data to be used(sequences examples)
    k: Is the length of the kmers
    m: Is the number of possible(maximum) mismatches allowed.
    """
    neighbours, kmer_set = None, None
    try:
        #Load the neighbors
        neighbours, kmer_set = load_neighbors(dataset, k, m)

    except:
        print('No file found, creating kmers neighbors')
        #Let Compute the neighbors
        file_name = 'neighbours_'+str(dataset)+'_'+str(k)+'_'+str(m)+'.p'
        # Please remember to put the address or location of your CSV file here in the following lines.
        if dataset==0:
            X_train = pd.read_csv("/content/Xtr.csv", sep=",", index_col=0).values
            X_test = pd.read_csv("/content/Xte.csv", sep=",", index_col=0).values
            kmer_set = create_kmer_set(X_train[:,0], k, kmer_set={})
            kmer_set = create_kmer_set(X_test[:,0], k, kmer_set)
            neighbours = get_neighbours(kmer_set, m)
            pickle.dump([neighbours, kmer_set], open(file_name, 'wb'))

    return neighbours, kmer_set

# Let's fix the hyperparameters to be used

In [ ]:
C = 1.0
k = 12
m = 2
list_k = [5,8,10,12,13,15]
list_m = [1,1,1,2,2,3]


#  SVM is the machine learning technique we used here:

# To support the decision rule of SVM in prediction based on the hyperplanes, let's modify the $y_{train}$  value.

$w$: is the weights matrix, $X_i$: is the input example, $b$: is the bias, and $y_i$: is the correct label correspond to $X_i$.

$y_i\in \{-1,1\}$

First class: $y_i(w^T X_i+b)\ge 1$ and the second is: $y_i(w^TX_i+b)\le -1$, while the hyperplane that separates two classes is: $(w^T X_i+b)= 0$


In [ ]:

Y_train = np.where(Y_train == 0, -1, 1)

# Creation of the neighbors, set of kmers, and mentioning of the kernel to use.

In [ ]:
kernel='mismatch'

neighbours, kmer_set = load_or_compute_neighbors(0,k,m)

No file found, creating kmers neighbors


100%|██████████| 712790/712790 [06:47<00:00, 1747.07it/s]


# We need to shuffle the data to make model adapt the new data due to how it will be arranged, this increase the model performace.

In [ ]:
shuffle=np.random.shuffle
shuffle = True  #Shuffle the data


In [ ]:
shuffling = np.random.permutation(len(X_train))
X_train = X_train[shuffling][:,0]
Y_train = Y_train[shuffling]

IndexError: ignored

# Prepare the test data matrices to be used by putting it into the right format.

In [ ]:

X_test = X_test[:,0]

# Setting the hyperparameter and kernel to use:

$C=1.0$ and kernel is: Mismatching kernel.

In [ ]:
print("The value of C used is:", C,'and kernel is:',kernel)
print("Let's apply SVM on dataset")



svm = SVM(kernel=MismatchKernel(k=k, m=m, neighbours=neighbours, kmer_set=kmer_set,normalize=True), C=C)



The value of C used is: 1.0 and kernel is: mismatch
Let's apply SVM on dataset


# Fit the data to the model

In [ ]:
svm.fit(X_train, Y_train)
prediction = svm.predict_classes(X_test)
pred = prediction.squeeze()


Computing the kernel Start


ValueError: ignored

# The predicted value

In [ ]:
print(pred)

NameError: ignored

# Let's bring back the label in terms of zero and one in order to compare it with the correct label used in the dataset.

In [ ]:
pred = np.where(pred == -1, 0, 1)
print('The predicted labels are:')
print('************************\n')
print(pred)

The predicted labels are:
************************

[0 1 1 0 1 1 1 0 1 1 0 0 0 1 0 1 1 1 1 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 1 0
 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 0 0 0 1 1 1 1 0 1 0 1 0 0 1 1 0 0 1 1 1 1 1
 1 0 0 1 1 0 0 1 0 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 0 1 0 0 1 0
 1 1 0 1 1 0 1 0 1 1 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 1 0 1 0 1 1 0 0 1 1
 0 1 0 0 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 1 1 1 1 1 0 0 0 0
 1 1 0 1 1 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1
 1 0 1 1 0 0 1 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 1
 1 1 1 1 0 0 0 0 1 1 0 1 0 1 0 1 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 0 0 0 1 0 1
 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0
 0 0 1 1 1 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 1 1
 1 1 0 1 1 1 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 1 1 1 0 1 1 0 0 0 1 1 1 1 0 1 1
 1 1 0 1 1 1 1 0 1 0 0 1 1 0 1 1 1 0 0 1 1 0 0 0

# Creation of the DataFrame for saving the predicted results

In [ ]:
pred_df = pd.DataFrame()
pred_df['Covid'] = pred
pred_df.index.name = 'Id'
pred_df.index += 1
pred_df.to_csv('CovID-19.csv', sep=',', header=True)

# Read the saved csv file that was named  CovID-19.

In [ ]:
covid_saved_data=pd.read_csv('/content/CovID-19.csv')
#Read the first 25 lines
covid_saved_data.head(25)

,Id,Covid
0,1,0
1,2,1
2,3,1
3,4,0
4,5,1
5,6,1
6,7,1
7,8,0
8,9,1
9,10,1
